In [20]:
# Импортируем библиотеки
import json
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS

In [21]:
# Функция, которая парсит mcc код и возвращает категорию компании
def get_type(org_name: str):
    # Уберем из названия компании все, что не является буквой или цифрой
    org_name = re.sub(r'[^a-zA-Zа-яА-Я0-9 ]', ' ', org_name)
    
    # Отправим запрос на сайт 
    request = requests.get(f'https://mcc-codes.ru/search/?q=' + '+'.join(org_name.split()))
    bs = BS(request.content, 'html.parser')
    objects = list(bs.findAll("a")) 
    org_types = []
    mcc_codes = []
    # Получим значения
    for i in objects:
        if i.text.isdigit():
            if len(str(int(i.text))) == 4:
                org_types.append(i['title'])
                mcc_codes.append(i.text)
                
    # Вернем данные
    if len(org_types) == 0:
        return {
            'code' : None,
            'type' : None
        }
    else: 
        return {
            'code' : int(max(mcc_codes, key= lambda x: mcc_codes.count(x))),
            'type' : max(org_types, key= lambda x: org_types.count(x))
        } 

In [22]:
# Откроем необходимые файлы
df = pd.read_csv("data.csv")
file = open("train_mcc_codes.json", "w", encoding="utf-8")

In [23]:
# Создадим словарь, в котором будем хранить информацию о категориях компаний
organisations = df['merchant_name'].unique()
data = {"count": len(organisations)}

# Пройдем по всем организациям
already = 0
print('0.00%')
for organisation in organisations:
    result = get_type(organisation)
    data[organisation] = result
    already += 1 
    print(str(round(100 * already / len(organisations), 2)) + '%')

0.00%
0.22%
0.44%
0.65%
0.87%
1.09%
1.31%
1.53%
1.74%
1.96%
2.18%
2.4%
2.61%
2.83%
3.05%
3.27%
3.49%
3.7%
3.92%
4.14%
4.36%
4.58%
4.79%
5.01%
5.23%
5.45%
5.66%
5.88%
6.1%
6.32%
6.54%
6.75%
6.97%
7.19%
7.41%
7.63%
7.84%
8.06%
8.28%
8.5%
8.71%
8.93%
9.15%
9.37%
9.59%
9.8%
10.02%
10.24%
10.46%
10.68%
10.89%
11.11%
11.33%
11.55%
11.76%
11.98%
12.2%
12.42%
12.64%
12.85%
13.07%
13.29%
13.51%
13.73%
13.94%
14.16%
14.38%
14.6%
14.81%
15.03%
15.25%
15.47%
15.69%
15.9%
16.12%
16.34%
16.56%
16.78%
16.99%
17.21%
17.43%
17.65%
17.86%
18.08%
18.3%
18.52%
18.74%
18.95%
19.17%
19.39%
19.61%
19.83%
20.04%
20.26%
20.48%
20.7%
20.92%
21.13%
21.35%
21.57%
21.79%
22.0%
22.22%
22.44%
22.66%
22.88%
23.09%
23.31%
23.53%
23.75%
23.97%
24.18%
24.4%
24.62%
24.84%
25.05%
25.27%
25.49%
25.71%
25.93%
26.14%
26.36%
26.58%
26.8%
27.02%
27.23%
27.45%
27.67%
27.89%
28.1%
28.32%
28.54%
28.76%
28.98%
29.19%
29.41%
29.63%
29.85%
30.07%
30.28%
30.5%
30.72%
30.94%
31.15%
31.37%
31.59%
31.81%
32.03%
32.24%
32.46%
32.68%
32.9

In [24]:
# Запишем данные в файл
json.dump(data, file, indent=4, ensure_ascii=False)
file.close()